<a href="https://colab.research.google.com/github/SarathiSoundarya/MPI_Codes/blob/main/MPI_Simpson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%writefile simps.c
#include<stdio.h>
#include<stdlib.h>
#include<mpi.h>

double eval(float x)
{
    return x*x; //the function to evaluate
}
float simpson(float *x, int localn)
{
    float localsum=0;
    localsum+=eval(x[0])+eval(x[localn-1]);
    for(int i=1;i<localn-1;i++)
    {
        if(i%2==0)
        {
          localsum+=(2*eval(x[i]));
        }
     else
     {
         localsum+=(4*eval(x[i]));
     }
    }
 return localsum;
}
int main(int argc, char ** argv)
{
 float  *x, totalsum,xo=0, xn=2, h=0.1;
 int i,n,index,rank,size,eleft;
 MPI_Status status;
 MPI_Init(&argc,&argv);
 MPI_Comm_rank(MPI_COMM_WORLD,&rank);
 MPI_Comm_size(MPI_COMM_WORLD,&size);
 int localn;
 float localsum=0;
 if(rank==0)
 {
  n=((xn-xo)/h)+1;
  x=(float*)malloc(n*sizeof(float));
  for(int i=0;i<=n;i++)
  { 
      x[i-1]=xo+(i*h); 
  }
  localn=n/size;
  for (i=1;i<size-1;i++)
  { 
      MPI_Send(&localn,1,MPI_INT,i,0,MPI_COMM_WORLD);
      index=i*localn;
      MPI_Send(&x[index],localn,MPI_FLOAT,i,0,MPI_COMM_WORLD);
  }
  index=i*localn;
  eleft=n-index;
  MPI_Send(&eleft,1,MPI_INT,i,0,MPI_COMM_WORLD);
  MPI_Send(&x[index],eleft,MPI_INT,i,0,MPI_COMM_WORLD);
  localsum=simpson(x,localn);
 }
 else
 {
 MPI_Recv(&localn,1,MPI_INT,0,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
 float xl[localn];
 MPI_Recv(&xl,localn,MPI_FLOAT,0,0,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
 localsum=simpson(xl,localn);
 }
 printf("localsum by Simpson's rule for rank %d is: %f \n",rank,localsum);
 MPI_Barrier(MPI_COMM_WORLD);
 MPI_Reduce(&localsum,&totalsum,1,MPI_FLOAT,MPI_SUM,0,MPI_COMM_WORLD);
 if(rank==0)
 {
     totalsum*=(h/3);
     printf("\ntotalsum is: %f",totalsum);
 }
 MPI_Finalize();
 return 0;
}

Overwriting simps.c


In [25]:
%%shell
mpicc simps.c -o simps
mpirun --allow-run-as-root -np 4 ./simps

localsum by Simpson's rule for rank 0 is: 1.240000 
localsum by Simpson's rule for rank 3 is: 47.450001 
localsum by Simpson's rule for rank 2 is: 20.440001 
localsum by Simpson's rule for rank 1 is: 7.840000 

totalsum is: 2.565667